In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os, io

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.losses import BinaryFocalCrossentropy
import tensorflow.keras.backend as K

import datetime
import matplotlib.pyplot as plt

In [2]:
BATCH = 16
RESIZE = 128

In [ ]:
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    y_pred_f = tf.clip_by_value(y_pred_f, smooth, 1)
    y_true_f = tf.clip_by_value(y_true_f, smooth, 1)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


# Combined Dice + Focal Loss
class DiceFocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.95, gamma=1.5):
        super().__init__()
        self.focal = BinaryFocalCrossentropy(alpha=alpha, gamma=gamma)

    def call(self, y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        dice_loss = 1 - dice_coefficient(y_true, y_pred)
        focal_loss = self.focal(y_true, y_pred)
        return dice_loss + 0.01 * focal_loss

In [ ]:
def load_tif_data(data_dir):
    images, masks = [], []
    for patient_dir in os.listdir(data_dir):
        patient_path = os.path.join(data_dir, patient_dir)
        if not os.path.isdir(patient_path):
            continue
        # Get all .tif files in the patient directory
        tif_files = [f for f in os.listdir(patient_path) if f.endswith('.tif') and 'mask' not in f]
        for img_file in tif_files:
            img_path = os.path.join(patient_path, img_file)
            images.append(img_path)
            # Construct corresponding mask filename
            mask_file = img_file.replace('.tif', '_mask.tif')
            mask_path = os.path.join(patient_path, mask_file)
            masks.append(mask_path)
            if not os.path.exists(mask_path):
                continue  # Skip if mask file doesn’t exist
    
    return pd.DataFrame({'img': images, 'mask': masks})

In [4]:
df = pd.read_csv('data_addrs.csv')

In [5]:
df

,img,mask
0,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
1,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
2,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
3,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
4,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
...,...,...
3924,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
3925,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
3926,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
3927,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...


In [6]:
x_tr, x_val = train_test_split(df, test_size=0.3, random_state=42)
x_val, x_ts = train_test_split(x_val, test_size=0.5, random_state=42)

print(f"Train: {len(x_tr)}, Val: {len(x_val)}, Test: {len(x_ts)}")

Train: 2750, Val: 589, Test: 590


In [7]:
def mask_aware_random_crop(image, mask, crop_size):
    """Randomly crop image and mask ensuring positive pixels remain (if exist)."""
    combined = tf.concat([image, mask], axis=-1)
    shape = tf.shape(combined)
    h, w = shape[0], shape[1]

    def crop_once():
        cropped = tf.image.random_crop(combined, size=[crop_size, crop_size, tf.shape(combined)[-1]])
        return cropped

    def condition(cropped):
        mask_crop = cropped[..., -1:]
        # Check if there are any positive pixels
        return tf.reduce_sum(mask_crop) <= 0.0

    # Try a few times to get a crop with non-empty mask
    def body(cropped):
        new_crop = crop_once()
        return tf.cond(condition(new_crop), lambda: new_crop, lambda: cropped)

    # Perform first crop
    cropped = crop_once()
    # Retry up to 3 times
    for _ in range(3):
        cropped = body(cropped)

    image_cropped = cropped[..., :-1]
    mask_cropped = cropped[..., -1:]
    return image_cropped, mask_cropped

In [8]:
def augment(image, mask):
    # Random flips
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_up_down(image)
        mask = tf.image.flip_up_down(mask)

    # Random rotation
    # k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    # image = tf.image.rot90(image, k)
    # mask = tf.image.rot90(mask, k)

    # Random brightness/contrast (image only)
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_contrast(image, 0.9, 1.1)

    # ✅ Mask-aware crop
    image, mask = mask_aware_random_crop(image, mask, crop_size=RESIZE)

    return image, mask

In [9]:
def _load_tif_image(path, ch):
    img = Image.open(path.decode())
    
    if int(ch) == 3:
        img = img.convert('RGB')
    else:
        img = img.convert('L')
    
    img = img.resize((RESIZE, RESIZE))
    img = np.array(img).astype(np.float32) / 255.0

    # If grayscale (2D), expand to [H, W, 1]
    if img.ndim == 2:
        img = np.expand_dims(img, axis=-1)
    return img


def load_image_and_mask(image_path, mask_path):
    # Load image
    image = tf.numpy_function(_load_tif_image, [image_path, 3], tf.float32)
    image.set_shape([RESIZE, RESIZE, 3])  # or [RESIZE, RESIZE, 1]

    # Load mask (assume single channel)
    mask = tf.numpy_function(_load_tif_image, [mask_path, 1], tf.float32)
    mask.set_shape([RESIZE, RESIZE, 1])

    return image, mask

In [10]:
trds = tf.data.Dataset.from_tensor_slices((x_tr['img'], x_tr['mask']))
vlds = tf.data.Dataset.from_tensor_slices((x_val['img'], x_val['mask']))
tsds = tf.data.Dataset.from_tensor_slices((x_ts['img'], x_ts['mask']))


trds = (trds
	.shuffle(1024)
	.map(load_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
	.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
	.cache()
	.batch(BATCH)
	.prefetch(tf.data.AUTOTUNE)
)

vlds = (vlds
	.shuffle(1024)
	.map(load_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
	.cache()
	.batch(BATCH)
	.prefetch(tf.data.AUTOTUNE)
)

tsds = (tsds
	.shuffle(1024)
	.map(load_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
	.cache()
	.batch(BATCH)
	.prefetch(tf.data.AUTOTUNE)
)

In [11]:
for a, b in tsds.take(1):
    print(a)
    print()
    print(b)
    

tf.Tensor(
[[[[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   ...
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   ...
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.00392157]
   ...
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  ...

  [[0.         0.         0.        ]
   [0.         0.01176471 0.        ]
   [0.         0.03137255 0.        ]
   ...
   [0.00784314 0.02745098 0.00392157]
   [0.         0.03529412 0.        ]
   [0.         0.03921569 0.        ]]

  [[0.         0.         0.      

In [12]:
# for img, mask in trds.take(3):
#     plt.subplot(1,2,1)
#     plt.imshow(img[0])
#     plt.subplot(1,2,2)
#     plt.imshow(mask[0,...,0], cmap='gray')
#     plt.show()

In [13]:
# Define TumorSegNet (same as before, included for completeness)
def TumorSegNet(input_shape=(256, 256, 3)):
    inputs = Input(input_shape, name='Input_Layer')
    # Encoder
    c1 = Conv2D(64, 3, activation='relu', padding='same', name='Conv1_1')(inputs)
    c1 = Conv2D(64, 3, activation='relu', padding='same', name='Conv1_2')(c1)
    p1 = MaxPooling2D((2, 2), name='MaxPool1')(c1)
    c2 = Conv2D(128, 3, activation='relu', padding='same', name='Conv2_1')(p1)
    c2 = Conv2D(128, 3, activation='relu', padding='same', name='Conv2_2')(c2)
    p2 = MaxPooling2D((2, 2), name='MaxPool2')(c2)
    c3 = Conv2D(256, 3, activation='relu', padding='same', name='Conv3_1')(p2)
    c3 = Conv2D(256, 3, activation='relu', padding='same', name='Conv3_2')(c3)
    p3 = MaxPooling2D((2, 2), name='MaxPool3')(c3)
    # Bottleneck
    c4 = Conv2D(512, 3, activation='relu', padding='same', name='Bottleneck_Conv1')(p3)
    c4 = Conv2D(512, 3, activation='relu', padding='same', name='Bottleneck_Conv2')(c4)
    c4 = Dropout(0.5, name='Bottleneck_Dropout')(c4)
    # Decoder
    u3 = UpSampling2D((2, 2), name='UpSample3')(c4)
    u3 = concatenate([u3, c3], name='Concat3')
    c5 = Conv2D(256, 3, activation='relu', padding='same', name='Conv5_1')(u3)
    c5 = Conv2D(256, 3, activation='relu', padding='same', name='Conv5_2')(c5)
    u2 = UpSampling2D((2, 2), name='UpSample2')(c5)
    u2 = concatenate([u2, c2], name='Concat2')
    c6 = Conv2D(128, 3, activation='relu', padding='same', name='Conv6_1')(u2)
    c6 = Conv2D(128, 3, activation='relu', padding='same', name='Conv6_2')(c6)
    u1 = UpSampling2D((2, 2), name='UpSample1')(c6)
    u1 = concatenate([u1, c1], name='Concat1')
    c7 = Conv2D(64, 3, activation='relu', padding='same', name='Conv7_1')(u1)
    c7 = Conv2D(64, 3, activation='relu', padding='same', name='Conv7_2')(c7)
    outputs = Conv2D(1, 1, activation='sigmoid', name='Output_Layer')(c7)
    return Model(inputs, outputs, name='TumorSegNet')

In [14]:
# Instantiate and compile model
model = TumorSegNet(input_shape=(RESIZE, RESIZE, 3))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss=DiceFocalLoss(),
            metrics=[dice_coefficient, 'accuracy'])

# TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
    profile_batch=0
)

# Custom callback to log sample predictions to TensorBoard
class PredictionLogger(tf.keras.callbacks.Callback):
    def __init__(self, val_images, val_masks):
        super(PredictionLogger, self).__init__()
        self.val_images = val_images[:5]  # Log first 5 validation images
        self.val_masks = val_masks[:5]

    def on_epoch_end(self, epoch, logs=None):
        preds = self.model.predict(self.val_images)
        preds = (preds > 0.5).astype(np.uint8)
        
        # Create figure with images, true masks, and predictions
        fig = plt.figure(figsize=(15, 5 * len(self.val_images)))
        for i in range(len(self.val_images)):
            plt.subplot(len(self.val_images), 3, i * 3 + 1)
            plt.imshow(self.val_images[i].squeeze(), cmap='gray')
            plt.title('MRI Image')
            plt.axis('off')
            plt.subplot(len(self.val_images), 3, i * 3 + 2)
            plt.imshow(self.val_masks[i].squeeze(), cmap='gray')
            plt.title('True Mask')
            plt.axis('off')
            plt.subplot(len(self.val_images), 3, i * 3 + 3)
            plt.imshow(preds[i].squeeze(), cmap='gray')
            plt.title(f'Predicted Mask (Epoch {epoch + 1})')
            plt.axis('off')
        
        # Log figure to TensorBoard
        writer = tf.summary.create_file_writer(log_dir)
        with writer.as_default():
            tf.summary.image("Validation Predictions", plot_to_image(fig), step=epoch)
        plt.close(fig)

# Convert matplotlib figure to TensorBoard image format
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

# Other callbacks
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss'),
    ModelCheckpoint('TumorSegNet_best.keras', save_best_only=True, monitor='val_dice_coefficient'),
    tensorboard_callback,
    # PredictionLogger(x_val['img'], masks_val)
]

# Train the model
history = model.fit(
    trds,
    validation_data=vlds,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 1348s 8s/step - accuracy: 0.9784 - dice_coefficient: 0.1562 - loss: 0.8451 - val_accuracy: 0.9877 - val_dice_coefficient: 0.4719 - val_loss: 0.5296
Epoch 2/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 2422s 14s/step - accuracy: 0.9876 - dice_coefficient: 0.5215 - loss: 0.4793 - val_accuracy: 0.9909 - val_dice_coefficient: 0.5590 - val_loss: 0.4423
Epoch 3/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 2388s 14s/step - accuracy: 0.9898 - dice_coefficient: 0.5761 - loss: 0.4246 - val_accuracy: 0.9900 - val_dice_coefficient: 0.5688 - val_loss: 0.4324
Epoch 4/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 2310s 13s/step - accuracy: 0.9913 - dice_coefficient: 0.6409 - loss: 0.3598 - val_accuracy: 0.9917 - val_dice_coefficient: 0.6018 - val_loss: 0.3994
Epoch 5/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 2313s 13s/step - accuracy: 0.9916 - dice_coefficient: 0.6558 - loss: 0.3448 - val_accuracy: 0.9922 - val_dice_coefficient: 0.6461 - val_loss: 0.3550


In [15]:
model.save('TumorSegNet_best.keras')